In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import glob

from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


In [2]:
class FaceGlassLoadDataset(Dataset):
    """Face Landmarks dataset."""

    # Read a PGM file and return a list of lists (each list is a line)
    def read_pgm(self, pgmf):
        assert pgmf.readline() == b'P5\n'
        (width, height) = [int(i) for i in pgmf.readline().split()]
        depth = int(pgmf.readline())
        assert depth <= 255

        raster = []
        for y in range(height):
            for y in range(width):
                raster.append(np.array(ord(pgmf.read(1))))
        return np.array(raster)
    
    def __init__(self, root_dir, transform=None):
        # Read all images to build dataset
        images = []
        labels = []
        
        for filename in glob.glob(root_dir + "*.pgm"):
            if "sunglasses" in filename:
                label = 1
            else:
                label = 0
            image = Image.open(filename)
            images.append(image)
            labels.append(label)
        
        
        
        self.imagesData = images        
        self.labelsData = labels
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.imagesData)

    def __getitem__(self, idx):
        sample = {'image': self.imagesData[idx], 'label': self.labelsData[idx]}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['label'] = torch.from_numpy(np.array(sample['label'], dtype='long')).long()           

        return sample

In [3]:
class FaceDirectionLoadDataset(Dataset):
    """Face Landmarks dataset."""

    # Read a PGM file and return a list of lists (each list is a line)
    def read_pgm(self, pgmf):
        assert pgmf.readline() == b'P5\n'
        (width, height) = [int(i) for i in pgmf.readline().split()]
        depth = int(pgmf.readline())
        assert depth <= 255

        raster = []
        for y in range(height):
            for y in range(width):
                raster.append(np.array(ord(pgmf.read(1))))
        return np.array(raster)
    
    def __init__(self, root_dir, transform=None):
        # Read all images to build dataset
        images = []
        labels = []
        
        for filename in glob.glob(root_dir + "*.pgm"):
            if "left" in filename:
                label = 0
            elif "right" in filename:
                label = 1
            elif "straight" in filename:
                label = 2
            elif "up" in filename:
                label = 3
            image = Image.open(filename)
            images.append(image)
            labels.append(label)
        
        
        
        self.imagesData = images        
        self.labelsData = labels
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.imagesData)

    def __getitem__(self, idx):
        sample = {'image': self.imagesData[idx], 'label': self.labelsData[idx]}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['label'] = torch.from_numpy(np.array(sample['label'], dtype='long')).long()           

        return sample

In [4]:
class FaceFeelLoadDataset(Dataset):
    """Face Landmarks dataset."""

    # Read a PGM file and return a list of lists (each list is a line)
    def read_pgm(self, pgmf):
        assert pgmf.readline() == b'P5\n'
        (width, height) = [int(i) for i in pgmf.readline().split()]
        depth = int(pgmf.readline())
        assert depth <= 255

        raster = []
        for y in range(height):
            for y in range(width):
                raster.append(np.array(ord(pgmf.read(1))))
        return np.array(raster)
    
    def __init__(self, root_dir, transform=None):
        # Read all images to build dataset
        images = []
        labels = []
        
        for filename in glob.glob(root_dir + "*.pgm"):
            if "angry" in filename:
                label = 0
            elif "happy" in filename:
                label = 1
            elif "neutral" in filename:
                label = 2
            elif "sad" in filename:
                label = 3
            image = Image.open(filename)
            images.append(image)
            labels.append(label)
        
        
        
        self.imagesData = images        
        self.labelsData = labels
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.imagesData)

    def __getitem__(self, idx):
        sample = {'image': self.imagesData[idx], 'label': self.labelsData[idx]}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
            sample['label'] = torch.from_numpy(np.array(sample['label'], dtype='long')).long()           

        return sample

In [5]:
preprocess = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


dataset = FaceFeelLoadDataset(root_dir='faces_4/*/', transform=preprocess)


batch_size = 4
validation_split = .1
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size, sampler=validation_sampler)


In [6]:
class Net(nn.Module):
    def __init__(self, num_classes=2):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=12)
        self.relu2 = nn.ReLU()

        self.pool1 = nn.MaxPool2d(kernel_size=2)       

        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=24)
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=24)
        self.relu4 = nn.ReLU()
        
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(num_features=48)
        self.relu5 = nn.ReLU()
        
        self.conv6 = nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(num_features=48)
        self.relu6 = nn.ReLU()
        
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        
        self.conv7 = nn.Conv2d(in_channels=48, out_channels=96, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(num_features=96)
        self.relu7 = nn.ReLU()
        
        self.conv8 = nn.Conv2d(in_channels=96, out_channels=96, kernel_size=3, stride=1, padding=1)
        self.bn8 = nn.BatchNorm2d(num_features=96)
        self.relu8 = nn.ReLU()        
        
        self.avgpool = nn.AvgPool2d (kernel_size = 4)
                       

        self.fc = nn.Linear(in_features= 96, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)

        output = self.pool1(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)

        output = self.conv4(output)
        output = self.bn4(output)
        output = self.relu4(output)
         
        output = self.pool2(output)
        
        output = self.conv5(output)
        output = self.bn5(output)
        output = self.relu5(output)
        
        output = self.conv6(output)
        output = self.bn6(output)
        output = self.relu6(output)  
        
        output = self.pool3(output)
        
        output = self.conv7(output)
        output = self.bn7(output)
        output = self.relu7(output)
        
        output = self.conv8(output)
        output = self.bn8(output)
        output = self.relu8(output)  
        
        output = self.avgpool(output)

        output = output.view(-1, 96)

        output = self.fc(output)

        return output


net = Net(4)



In [7]:
#Let’s use a Classification Cross-Entropy loss and SGD with momentum.
from torch.optim import Adam

criterion = nn.CrossEntropyLoss()

optimizer = Adam(net.parameters(), lr=0.001, weight_decay=0.0001) #optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)

In [35]:
checkpoint = torch.load("modelo_feel")
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

net.eval() # - or - model.train()

Net(
  (conv1): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(12, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (conv4): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): Ba

In [37]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
       
        # get the inputs
        inputs, labels = data["image"], data["label"]
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 1.386
[2,   100] loss: 1.388
[3,   100] loss: 1.388
[4,   100] loss: 1.386
[5,   100] loss: 1.386
[6,   100] loss: 1.386
[7,   100] loss: 1.385
[8,   100] loss: 1.387
[9,   100] loss: 1.387
[10,   100] loss: 1.388
[11,   100] loss: 1.387
[12,   100] loss: 1.385
[13,   100] loss: 1.387
[14,   100] loss: 1.386
[15,   100] loss: 1.385
[16,   100] loss: 1.386
[17,   100] loss: 1.387
[18,   100] loss: 1.386
[19,   100] loss: 1.387
[20,   100] loss: 1.387
[21,   100] loss: 1.386
[22,   100] loss: 1.387
[23,   100] loss: 1.385
[24,   100] loss: 1.386
[25,   100] loss: 1.386
[26,   100] loss: 1.386
[27,   100] loss: 1.386
[28,   100] loss: 1.385
[29,   100] loss: 1.385
[30,   100] loss: 1.385
[31,   100] loss: 1.387
[32,   100] loss: 1.387
[33,   100] loss: 1.387
[34,   100] loss: 1.386
[35,   100] loss: 1.387
[36,   100] loss: 1.385
[37,   100] loss: 1.387
[38,   100] loss: 1.386
[39,   100] loss: 1.387
[40,   100] loss: 1.387
[41,   100] loss: 1.387
[42,   100] loss: 1.387
[

In [38]:
torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss}, "modelo_feel")

In [51]:
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        images, labels = data["image"], data["label"]
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 25 %
